# Extração - Limpeza e Tratamento dos Dados

In [1]:
!pip install inflection
!pip install streamlit-folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.0/326.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import inflection
import plotly.express as px
import folium
from folium.plugins import MarkerCluster
from streamlit_folium import folium_static


In [3]:
#Importando os dados
dados_principal = pd.read_csv('/content/drive/MyDrive/ComunidadeDS/Dados/zomato.csv')
dados = dados_principal.copy()

In [4]:
COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}

def country_name(country_id):
  return COUNTRIES[country_id]

#Criação do Tipo de Categoria de Comida
def create_price_tye(price_range):
  if price_range == 1:
    return "cheap"
  elif price_range == 2:
    return "normal"
  elif price_range == 3:
    return "expensive"
  else:
    return "gourmet"


#Criação do nome das Cores
COLORS = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred",
}
def color_name(color_code):
  return COLORS[color_code]


#Renomear as colunas do DataFrame
def renomear_colunas(df_dados):
  df = df_dados.copy()
  title = lambda x: inflection.titleize(x)
  snakecase = lambda x: inflection.underscore(x)
  spaces = lambda x: x.replace(" ", "")

  cols_old = list(df.columns)
  cols_old = list(map(title, cols_old))
  cols_old = list(map(spaces, cols_old))

  cols_new = list(map(snakecase, cols_old))
  df.columns = cols_new

  return df


In [5]:
#Renomear Colunas
dados = renomear_colunas(dados)

#Deletando os registros NaN, somente a coluna Cuisines possuia esses registros.
dados = dados[~dados.isna().any(axis=1)]

#Deletando a Coluna.: Switch to order menu, ela possui somente o valor 0(zero) em todos os registros.
dados.drop('switch_to_order_menu', axis=1, inplace=True)

#Deletando os registros Duplicados.
dados.drop_duplicates(inplace=True)

dados["cuisines"] = dados.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])

#Resetando o indice
dados.reset_index(drop=True, inplace=True)

dados['country_name'] = dados['country_code'].apply(lambda x: country_name(x))

dados["color_name"] = dados.loc[:, "rating_color"].apply(lambda x: color_name(x))


In [6]:
dados

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,has_table_booking,has_online_delivery,is_delivering_now,price_range,aggregate_rating,rating_color,rating_text,votes,country_name,color_name
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,1,0,0,3,4.6,3F7E00,Excellent,619,Philippines,darkgreen
1,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,...,0,0,0,4,4.7,3F7E00,Excellent,469,Philippines,darkgreen
2,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,...,0,0,0,3,4.4,5BA829,Very Good,867,Philippines,green
3,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,0,0,0,3,4.4,5BA829,Very Good,858,Philippines,green
4,6304833,Manam,162,Makati City,"Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 2, San Lorenzo, Makati City, Makati ...",121.020380,14.552351,Filipino,...,0,0,0,3,4.7,3F7E00,Excellent,930,Philippines,darkgreen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6924,5912546,Eataly,208,İstanbul,"Zorlu Center AVM, Köprü Katı, Levazım Mahalles...","Zorlu Center AVM, Levazım, Beşiktaş","Zorlu Center AVM, Levazım, Beşiktaş, İstanbul",29.017326,41.065322,Italian,...,0,0,0,4,4.3,5BA829,Very Good,1367,Turkey,green
6925,5913006,Tarihi Çınaraltı Aile Çay Bahçesi,208,İstanbul,"Çengelköy Mahallesi, Çınaraltı Camii Sokak, No...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.052620,41.050280,Fast Food,...,0,0,0,2,4.5,3F7E00,Excellent,1172,Turkey,darkgreen
6926,5923535,Boon Cafe & Restaurant,208,İstanbul,"Çengelköy Mahallesi, Çengelköy Caddesi, Kara S...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.052623,41.050717,Restaurant Cafe,...,0,0,0,4,4.2,5BA829,Very Good,1160,Turkey,green
6927,5914190,Kanaat Lokantası,208,İstanbul,"Sultantepe Mahallesi, Selmani Pak Caddesi, No ...",Üsküdar Merkez,"Üsküdar Merkez, İstanbul",29.016590,41.025741,Home-made,...,0,0,0,3,4.0,5BA829,Very Good,770,Turkey,green


# 1 - Geral

## 1.1 - Quantos restaurantes únicos estão registrados?

In [7]:
dados['restaurant_id'].nunique()

6929

## 1.2 - Quantos países únicos estão registrados?

In [8]:
dados['country_code'].nunique()

15

## 1.3 - Quantas cidades únicas estão registradas?

In [9]:
dados['city'].nunique()

125

## 1.4 - Qual o total de avaliações feitas?

In [10]:
dados['votes'].sum()

4194533

## 1.5 - Qual o total de tipos de culinária registrados?

In [11]:
dados['cuisines'].nunique()


165

## 1.6 - Mapa

In [12]:
def gera_grafico_mapa(dataframe):
    f = folium.Figure(width=1920, height=1080)

    m = folium.Map(max_bounds=True).add_to(f)

    marker_cluster = MarkerCluster().add_to(m)

    for _, line in dataframe.iterrows():

        name = line["restaurant_name"]
        price_for_two = line["average_cost_for_two"]
        cuisine = line["cuisines"]
        currency = line["currency"]
        rating = line["aggregate_rating"]
        color = f'{line["color_name"]}'

        html = "<p><strong>{}</strong></p>"
        html += "<p>Price: {},00 ({}) para dois"
        html += "<br />Type: {}"
        html += "<br />Aggragate Rating: {}/5.0"
        html = html.format(name, price_for_two, currency, cuisine, rating)

        popup = folium.Popup(
            folium.Html(html, script=True),
            max_width=500,
        )

        folium.Marker(
            [line["latitude"], line["longitude"]],
            popup=popup,
            icon=folium.Icon(color=color, icon="home", prefix="fa"),
        ).add_to(marker_cluster)

    folium_static(m, width=1024, height=768)

selected_countries = ['Brazil', 'Canada', 'Australia', 'Indonesia']
map_df = dados.loc[dados["country_name"].isin(selected_countries), :]
gera_grafico_mapa(map_df)

2024-04-04 17:45:27.971 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


# 2 - Pais

## 2.1 - Qual o nome do país que possui mais cidades registradas?

In [13]:
indice_pais = dados[['country_code','city']].groupby(['country_code']).nunique().reset_index()['city'].idxmax()

df_aux = dados[['country_name','city']].groupby(['country_name']).nunique().sort_values('city', ascending=False).reset_index()


df_aux.loc[indice_pais, ['country_name', 'city']]

df_aux

px.bar(df_aux, x='country_name', y='city')






## 2.2 - Qual o nome do país que possui mais restaurantes registrados?

In [14]:
indice_rest = dados[['country_code', 'restaurant_id']].groupby(['country_code']).count().reset_index()['restaurant_id'].idxmax()

df_aux = dados[['country_name', 'restaurant_id']].groupby(['country_name']).count().sort_values('restaurant_id', ascending=False).reset_index()

df_aux.loc[indice_rest, :]

df_aux

px.bar(df_aux, x='country_name', y='restaurant_id', labels={'restaurant_id': 'Quantidade de Restaurantes', 'country_name': 'Países'}, title='Quantidade de Restaurantes Registrados por País')


## 2.3 - Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?

In [15]:
df_aux = dados[dados['price_range'] == 4][['country_code', 'restaurant_name']].groupby(['country_code']).nunique().sort_values('restaurant_name', ascending=False).reset_index()

df_aux

,country_code,restaurant_name
0,216,390
1,189,195
2,1,171
3,30,141
4,214,87
5,148,78
6,215,75
7,184,58
8,37,53
9,208,53


## 2.4 - Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [16]:
dados[['country_code','cuisines']].groupby(['country_code']).nunique().sort_values('cuisines', ascending=False).reset_index()

,country_code,cuisines
0,1,77
1,216,74
2,215,52
3,214,46
4,14,43
5,30,43
6,37,42
7,189,42
8,148,37
9,166,31


## 2.5 - Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [17]:
dados[['country_code','votes']].groupby(['country_code']).sum().sort_values('votes', ascending=False)

,votes
country_code,
1,2800164
216,522348
214,177564
14,130357
37,105018
208,100193
94,89026
189,81939
148,52532


## 2.6 - Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [18]:
dados[['country_code','is_delivering_now', 'restaurant_id']].groupby(['country_code','is_delivering_now']).count()

dados[dados['is_delivering_now'] == 1][['country_code','restaurant_id']].groupby(['country_code']).count().sort_values('restaurant_id', ascending=False).reset_index()

,country_code,restaurant_id
0,1,1150
1,214,52
2,162,7
3,166,3


## 2.7 - Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [19]:
dados[dados['has_table_booking'] == 1][['country_code','restaurant_id']].groupby(['country_code']).count().sort_values('restaurant_id', ascending=False).reset_index()

,country_code,restaurant_id
0,1,256
1,215,55
2,14,29
3,162,22
4,148,19
5,94,14
6,214,10
7,166,4
8,208,4
9,189,3


## 2.8 - Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [20]:
df_aux = dados[['country_name','votes']].groupby(['country_name']).agg(Media_avaliacao=('votes', 'mean')).round(3).sort_values('Media_avaliacao', ascending=False).reset_index()

px.bar(df_aux, x='country_name', y='Media_avaliacao')

## 2.9 - Qual o nome do país que possui, na média, a maior nota média registrada?

In [21]:
df_aux = dados[['country_code', 'aggregate_rating']].groupby(['country_code']).agg(Media_nota=('aggregate_rating', 'mean')).round(2).sort_values('Media_nota', ascending=False).reset_index()

df_aux.columns = ['Pais', 'Media_nota']

df_aux


,Pais,Media_nota
0,94,4.60
1,162,4.46
2,184,4.44
3,216,4.40
4,14,4.37
5,37,4.32
6,208,4.31
7,166,4.24
8,148,4.16
9,189,4.06


## 2.10 - Qual o nome do país que possui, na média, a menor nota média registrada?

In [22]:
df_aux = dados[['country_code', 'aggregate_rating']].groupby(['country_code']).agg(Media_nota=('aggregate_rating', 'mean')).round(2).sort_values('Media_nota').reset_index()

df_aux.columns = ['Pais', 'Media_nota']

df_aux

,Pais,Media_nota
0,30,3.32
1,214,4.02
2,1,4.04
3,215,4.04
4,189,4.06
5,191,4.06
6,148,4.16
7,166,4.24
8,208,4.31
9,37,4.32


## 2.11 - Qual a média de preço de um prato para dois por país?

In [23]:
df_aux = dados[['country_name', 'average_cost_for_two']].groupby(['country_name']).agg(Media_preco_prato=('average_cost_for_two', 'mean')).round(2).sort_values('Media_preco_prato', ascending=False).reset_index()

px.bar(df_aux, x='country_name', y='Media_preco_prato')

# 3 - Cidade

## 3.1 - Qual o nome da cidade que possui mais restaurantes registrados?

In [24]:
df_aux = dados[['country_name','city','restaurant_id']].groupby(['country_name','city']).agg(Qtde_restaurantes=('restaurant_id', 'count')).sort_values('Qtde_restaurantes', ascending=False).reset_index()

df_aux = df_aux.loc[0:50, :]



px.bar(df_aux, x='city', y='Qtde_restaurantes', color='country_name', labels={'country_name': 'País'})
#df_aux

## 3.2 - Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [25]:
df_aux = dados[['city', 'aggregate_rating']].groupby(['city']).agg(Media_nota=('aggregate_rating', 'mean')).round(2).sort_values('Media_nota', ascending=False).reset_index()

df_aux = df_aux[df_aux['Media_nota'] >= 4]
df_aux = df_aux.loc[0:6, :]

df_aux

,city,Media_nota
0,Muntinlupa City,4.90
1,Zirakpur,4.70
2,Ghaziabad,4.70
3,Inner City,4.65
4,New York City,4.61
5,London,4.61
6,Jakarta,4.61


## 3.3 - Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 3.5?

In [26]:
df_aux = dados[['city','aggregate_rating']].groupby(['city']).agg(Media_nota=('aggregate_rating', 'mean')).round(2).sort_values('Media_nota').reset_index()
df_aux = df_aux[df_aux['Media_nota'] <= 3.5]

df_aux

px.bar(df_aux, x='city', y='Media_nota', text='Media_nota',
       labels={'city': 'Cidades',
               'Media_nota': 'Média de Notas dos Restaurantes'})

## 3.4 - Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [27]:
dados[['city', 'average_cost_for_two']].groupby(['city']).agg(Media_preco=('average_cost_for_two', 'mean')).round(2).sort_values('Media_preco', ascending=False).reset_index()

,city,Media_preco
0,Adelaide,416734.13
1,Jakarta,309605.26
2,Tangerang,180000.00
3,Bogor,175000.00
4,Pasay City,4000.00
...,...,...
120,Calgary,32.50
121,Los Angeles,30.00
122,Denver,29.92
123,Phoenix,28.00


## 3.5 - Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [28]:
df_aux = dados[['city','cuisines']].groupby(['city']).agg(Qtde_culinaria=('cuisines', 'nunique')).sort_values('Qtde_culinaria', ascending=False).reset_index()

df_aux = df_aux.loc[0:9, :]

px.bar(df_aux, x='city', y='Qtde_culinaria', text='Qtde_culinaria',
       labels={'city': 'Cidades',
               'Qtde_culinaria': 'Quantidade de Culinárias'})

## 3.6 - Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [29]:
dados[dados['has_table_booking'] == 1][['city', 'restaurant_id']].groupby(['city']).agg(Qtde_rest_reserva=('restaurant_id', 'count')).sort_values('Qtde_rest_reserva', ascending=False).reset_index()

,city,Qtde_rest_reserva
0,Bangalore,42
1,Chennai,28
2,Pune,25
3,Mumbai,18
4,Brisbane,17
...,...,...
64,Pasig City,1
65,Fujairah,1
66,Nashik,1
67,Johannesburg,1


## 3.7 - Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [30]:
dados[dados['is_delivering_now'] == 1][['city', 'restaurant_id']].groupby(['city']).agg(Qtde_rest_entrega=('restaurant_id', 'count')).sort_values('Qtde_rest_entrega', ascending=False).reset_index()

,city,Qtde_rest_entrega
0,Vadodara,48
1,Amritsar,48
2,Aurangabad,47
3,Ludhiana,46
4,Bhopal,46
5,Dehradun,45
6,Ranchi,42
7,Jaipur,42
8,Ahmedabad,41
9,Varanasi,40


## 3.8 - Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [31]:
dados[dados['has_online_delivery'] == 1][['city','restaurant_id']].groupby(['city']).agg(Qtde_rest_ped_online=('restaurant_id', 'count')).sort_values('Qtde_rest_ped_online', ascending=False).reset_index()

,city,Qtde_rest_ped_online
0,Bhopal,75
1,Vadodara,74
2,Abu Dhabi,71
3,Sharjah,71
4,Nagpur,69
5,Aurangabad,69
6,Patna,69
7,Ranchi,69
8,Coimbatore,68
9,Allahabad,68


# 4 - Restaurantes

## 4.1 - Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [32]:
#dados[['restaurant_id', 'votes']].groupby(['restaurant_id']).agg(Qtde_avaliacao=('votes','sum')).sort_values('Qtde_avaliacao', ascending=False)

indice_rest = dados[['restaurant_id', 'votes']]['votes'].idxmax()
dados.loc[indice_rest, ['restaurant_id', 'restaurant_name', 'votes']]


restaurant_id         90896
restaurant_name    Bawarchi
votes                 41333
Name: 3678, dtype: object

## 4.2 - Qual o nome do restaurante com a maior nota média?

In [33]:
df_aux = dados[['restaurant_id', 'restaurant_name', 'aggregate_rating']].groupby(['restaurant_id', 'restaurant_name']).agg(Maior_avaliacao=('aggregate_rating', 'max')).sort_values(['Maior_avaliacao', 'restaurant_id'], ascending=[False, True]).reset_index()
df_aux


,restaurant_id,restaurant_name,Maior_avaliacao
0,7528,Indian Grill Room,4.9
1,8913,Pirates of Grill,4.9
2,10021,Darshan,4.9
3,20842,Barbeque Nation,4.9
4,56618,AB's - Absolute Barbecues,4.9
...,...,...,...
6924,18994548,V. Krishnan Bakery,0.0
6925,18997511,Jain Restaurant,0.0
6926,19034826,Republic of Chicken,0.0
6927,19035941,City Point,0.0


## 4.3 - Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?


In [34]:
dados[['restaurant_id','restaurant_name', 'average_cost_for_two']].sort_values('average_cost_for_two', ascending=False)

,restaurant_id,restaurant_name,average_cost_for_two
356,16608070,d'Arry's Verandah Restaurant,25000017
5567,7403667,The Café - Hotel Mulia,1200000
5603,7423164,GIA Restaurant & Bar,700000
5573,7425027,Shabu-Shabu Shaburi,600000
5562,7402935,SKYE,600000
...,...,...,...
1467,16665420,The Albion Rooms,0
5898,6901412,Nosh and Quaff,0
6644,18604603,Wok n Roll,0
1515,18196379,tbsp.,0


## 4.4 - Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [35]:
dados[dados['cuisines'] == 'Brazilian'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True])

,restaurant_id,restaurant_name,aggregate_rating
117,6600100,Loca Como tu Madre,0.0
96,6600119,Café Savana,0.0
116,6600124,Inácia Poulet Rôti,0.0
139,6600456,Galeteria Beira Lago,0.0
120,6601251,Severina,0.0
...,...,...,...
1020,16881352,Fogo de Chao Brazilian Steakhouse,4.8
191,7300955,Braseiro da Gávea,4.9
230,7302898,Aprazível,4.9
2212,18403563,Texas de Brazil,4.9


## 4.5 - Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [36]:
dados[(dados['cuisines'] == 'Brazilian') & (dados['country_code'] == 30)][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating','restaurant_id'], ascending=[False, True])

,restaurant_id,restaurant_name,aggregate_rating
191,7300955,Braseiro da Gávea,4.9
230,7302898,Aprazível,4.9
317,6709740,Mocotó,4.8
197,7300515,Garota de Ipanema,4.8
165,7301525,Fogo de Chão,4.8
199,7300483,Zazá Bistrô Tropical,4.7
235,7302859,Aconchego Carioca,4.7
318,6703176,Veloso,4.6
195,7300521,Balada Mix,4.6
316,6709580,Esquina Mocotó,4.5


## 4.6 - Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [37]:
qtde_rest_ped_online = dados[dados['has_online_delivery'] == 1]['restaurant_id'].count()
qtde_rest_ped_online_aval   = dados[(dados['has_online_delivery'] == 1) & (dados['votes'] >= 1)]['restaurant_id'].count()
qtde_rest_ped_online_s_aval = dados[(dados['has_online_delivery'] == 1) & (dados['votes'] == 0)]['restaurant_id'].count()

perc_ped_c_aval = round((qtde_rest_ped_online_aval / qtde_rest_ped_online)*100, 2)
perc_ped_s_aval = round((qtde_rest_ped_online_s_aval / qtde_rest_ped_online)*100, 2)

print(f'O Total de restaurantes que fazem pedidos Online são.: {qtde_rest_ped_online}')
print('='*80)
print(f'O Total de restaurantes que fazem pedidos Online e possuem avaliação é.: {qtde_rest_ped_online_aval} ')
print(f'E seu percentual é de.: {perc_ped_c_aval} %')
print('='*40)
print(f'O Total de restaurantes que fazem pedidos Online e não possuem avaliação é.: {qtde_rest_ped_online_s_aval}')
print(f'E seu percentual é de.: {perc_ped_s_aval} %')
print('='*80)

if perc_ped_c_aval > perc_ped_s_aval:
  resp = 'Sim'
else:
  resp = 'Não'

print('4.6 - Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?')
print(f'R.: {resp}')

O Total de restaurantes que fazem pedidos Online são.: 2428
O Total de restaurantes que fazem pedidos Online e possuem avaliação é.: 2426 
E seu percentual é de.: 99.92 %
O Total de restaurantes que fazem pedidos Online e não possuem avaliação é.: 2
E seu percentual é de.: 0.08 %
4.6 - Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?
R.: Sim


## 4.7 - Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [38]:
df_aux = dados[['has_table_booking','average_cost_for_two']].groupby(['has_table_booking']).agg(Media_preco=('average_cost_for_two', 'mean')).round(2).reset_index()

print('4.7 - Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?')
print('Resp.: Sim')

df_aux


4.7 - Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?
Resp.: Sim


,has_table_booking,Media_preco
0,0,3488.60
1,1,69998.42


## 4.8 - Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [39]:
indice_cozinhas_pais = (dados['cuisines'].isin(['Japanese','BBQ'])) & (dados['country_code'] == 216)
df_aux = dados[indice_cozinhas_pais][['cuisines', 'average_cost_for_two']].groupby(['cuisines']).agg(Media_prato=('average_cost_for_two', 'mean')).round(2).reset_index()

preco_medio_bbq = df_aux[df_aux['cuisines'] == 'BBQ']['Media_prato'].values[0]
preco_medio_japa = df_aux[df_aux['cuisines'] == 'Japanese']['Media_prato'].values[0]

if preco_medio_japa > preco_medio_bbq:
  resp = 'Sim'
else:
  resp = 'Não'

print('4.8 - Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?')
print(f'Resp.: {resp}')

df_aux


4.8 - Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?
Resp.: Sim


,cuisines,Media_prato
0,BBQ,39.64
1,Japanese,56.41


# 5 - Culinária

## 5.1 - Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [40]:
dados[dados['cuisines'] == 'Italian'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])

,restaurant_id,restaurant_name,aggregate_rating
4995,10021,Darshan,4.9
6651,6501298,Cafe Del Sol Classico,4.9
5810,7100171,Ombra,4.9
6054,7700796,Celino's,4.9
337,16587684,Andre's Cucina & Polenta Bar,4.9
...,...,...,...
6195,6800666,Le Delicatezze Di Bruno,0.0
220,7304910,Bene - Sheraton Rio Hotel,0.0
170,18334183,La Bocca Bar e Trattoria,0.0
6222,18756343,The Pasta Factory,0.0


## 5.2 - Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [41]:
dados[dados['cuisines'] == 'Italian'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True])

,restaurant_id,restaurant_name,aggregate_rating
138,6600542,Avenida Paulista,0.0
278,6714499,Ristorantino,0.0
297,6715707,Più,0.0
6195,6800666,Le Delicatezze Di Bruno,0.0
220,7304910,Bene - Sheraton Rio Hotel,0.0
...,...,...,...
1755,17156258,Cerroni's Purple Garlic,4.9
1769,17156955,Guillermo's,4.9
1348,17242351,Central Grocery,4.9
3796,18209498,Zolocrust - Hotel Clarks Amer,4.9


## 5.3 - Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?

In [42]:
dados[dados['cuisines'] == 'American'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])

,restaurant_id,restaurant_name,aggregate_rating
6123,6102616,Burger & Lobster,4.9
6228,6801374,Solita,4.9
694,16631515,OEB Breakfast Co.,4.9
790,16737455,Portillo's Hot Dogs,4.9
1410,16776778,S'MAC,4.9
...,...,...,...
5850,7100971,Thunderbird Cafe,3.2
6677,6502573,Hard Rock Cafe,2.8
228,7303748,Bob's,2.6
6625,18445965,Guy Fieri's Kitchen & Bar,2.2


## 5.4 - Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [43]:
dados[dados['cuisines'] == 'American'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True])

,restaurant_id,restaurant_name,aggregate_rating
6221,18756337,Alston Bar & Beef,0.0
6625,18445965,Guy Fieri's Kitchen & Bar,2.2
228,7303748,Bob's,2.6
6677,6502573,Hard Rock Cafe,2.8
5850,7100971,Thunderbird Cafe,3.2
...,...,...,...
1861,17069832,Hodad's,4.9
1868,17070293,Kono's Surf Club Cafe,4.9
1802,17153940,The Magnolia Pancake Haus,4.9
6309,18416901,The Cheesecake Factory,4.9


## 5.5 - Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [44]:
dados[dados['cuisines'] == 'Arabian'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])

,restaurant_id,restaurant_name,aggregate_rating
3647,18535007,Mandi@36,4.7
2375,5600517,Madfoon Al Khaimah,4.5
6336,6200166,Wok of Fame,4.5
3031,18770281,Three Kings,4.5
2235,201824,Aroos Damascus,4.4
2414,5601340,Aroos Damascus,4.4
2276,209723,Azkadenya,4.3
2180,18427358,Azkadenya,4.3
2282,207265,Awani,4.2
3967,902109,Sheba Kuzhimandhi,4.2


## 5.6 - Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [45]:
dados[dados['cuisines'] == 'Arabian'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True])

,restaurant_id,restaurant_name,aggregate_rating
310,6706513,Raful,0.0
155,6601535,Empório Árabe,2.7
135,6600547,Salim Sou Eu,3.1
4767,18698592,V Empire,3.2
4733,18698769,Abad Hot Chicken,3.2
2348,16540460,Murjan - Nour Arjaan by Rotana,3.3
2324,16541428,Bon Appetit,3.3
106,6600939,Libanus,3.4
2351,16540483,Breeze - Radisson Blu Resort,3.4
2350,16540485,Al Nokhada - Radisson Blu Resort,3.4


## 5.7 - Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?

In [46]:
dados[dados['cuisines'] == 'Japanese'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])


,restaurant_id,restaurant_name,aggregate_rating
6109,6107336,Sushi Samba,4.9
6134,6116563,Chotto Matte,4.9
31,6309831,Ramen Yushoken,4.9
16,6316125,Mendokoro Ramenba,4.9
20,6318506,Ooma,4.9
...,...,...,...
107,6600379,Sushi Loko,2.9
103,6600314,Haná,2.7
168,7304077,Hachiko,2.5
167,7300930,Orienthai,2.4


## 5.8 - Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [47]:
dados[dados['cuisines'] == 'Japanese'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True])

,restaurant_id,restaurant_name,aggregate_rating
114,6600203,Banzai Sushi,0.0
167,7300930,Orienthai,2.4
168,7304077,Hachiko,2.5
103,6600314,Haná,2.7
107,6600379,Sushi Loko,2.9
...,...,...,...
5622,18615392,WAKI Japanese BBQ Dining,4.9
67,18695427,Mendokoro Ramenba,4.9
5552,18793437,Gyu - Kaku Japanese BBQ,4.9
5553,18880385,Gyu - Kaku Japanese BBQ,4.9


## 5.9 - Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?

In [48]:
dados[dados['cuisines'] == 'Home-made'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])


,restaurant_id,restaurant_name,aggregate_rating
6927,5914190,Kanaat Lokantası,4.0
6781,6007184,GurMekan Restaurant,3.7


## 5.10 - Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?

In [49]:
dados[dados['cuisines'] == 'Home-made'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True])

,restaurant_id,restaurant_name,aggregate_rating
6781,6007184,GurMekan Restaurant,3.7
6927,5914190,Kanaat Lokantası,4.0


## 5.11 - Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [50]:
dados[['average_cost_for_two', 'cuisines']].groupby(['cuisines']).agg(Media_prato=('average_cost_for_two', 'mean')).round(2).sort_values('Media_prato', ascending=False)

,Media_prato
cuisines,
Modern Australian,1470693.06
Western,333333.33
Indonesian,200000.00
Sunda,200000.00
Dimsum,150000.00
...,...
Donuts,18.00
Russian,15.00
Polish,10.00


## 5.12 - Top 10 Melhores Tipos de Culinárias

In [51]:
df_aux = dados[['cuisines', 'aggregate_rating']].groupby(['cuisines']).agg(Media_nota=('aggregate_rating', 'mean')).round(2).sort_values('Media_nota', ascending=False).reset_index()

df_aux = df_aux.head(10)

fig = px.bar(df_aux, x='cuisines', y='Media_nota', text='Media_nota',
             labels={'cuisines': 'Tipo de Culinária',
                     'Media_nota': 'Média de Avaliações'})

fig.update_layout(title={'text': 'Top 10 Melhores Tipos de Culinárias',
                         'xanchor': 'center',
                         'x': 0.5},
                  title_font_size=20)

fig






## 5.13 - Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [52]:
indice_ped_online_entrega = (dados['has_online_delivery'] == 1) & (dados['is_delivering_now'] == 1)
dados[indice_ped_online_entrega][['cuisines', 'restaurant_id']].groupby(['cuisines']).agg(Qtde_restaurante=('restaurant_id', 'count')).sort_values('Qtde_restaurante', ascending=False)

,Qtde_restaurante
cuisines,
North Indian,317
Cafe,131
South Indian,79
Burger,76
Fast Food,72
Pizza,55
Chinese,49
Biryani,38
Italian,35


## 5.14 - Top 10 Piores Tipos de Culinárias

In [53]:
indice_culinaria = dados['aggregate_rating'] > 0
df_aux = dados[indice_culinaria][['cuisines', 'aggregate_rating']].groupby(['cuisines']).agg(Media_nota=('aggregate_rating', 'mean')).round(2).sort_values('Media_nota', ascending=True).reset_index()

df_aux = df_aux.head(10)

fig = px.bar(df_aux, x='cuisines', y='Media_nota', text='Media_nota',
             labels={'cuisines': 'Tipo de Culinária',
                     'Media_nota': 'Média de Avaliações'})

fig.update_layout(title={'text': 'Top 10 Piores Tipos de Culinárias',
                         'xanchor': 'center',
                         'x': 0.5},
                  title_font_size=20)

fig

## 5.15 - Top Restaurantes

In [54]:
culinarias = ['Italian','American','Arabian','Japanese','Brazilian']
colunas = ['restaurant_id', 'restaurant_name','country_name','city','cuisines','average_cost_for_two','currency','aggregate_rating','votes']


paises = dados['country_name'].unique()

lista_restaurantes = []
for pais in paises:
  #indice_culinaria = dados['cuisines'].isin(culinarias)
  indice_pais = dados['country_name'] == pais
  df_aux = dados.loc[indice_pais, colunas].sort_values(["aggregate_rating", "restaurant_id"], ascending=[False, True]).head(10)
  lista = list(df_aux['restaurant_id'].index.values)
  lista_restaurantes.extend(lista)


print(type(lista_restaurantes))
print(lista_restaurantes)
print(len(lista_restaurantes))

df_aux = dados.loc[lista_restaurantes, colunas].sort_values(["country_name", "aggregate_rating", "restaurant_id"], ascending=[True, False, True])

styles = [dict(selector="caption",
                       props=[("text-align", "center"),
                              ("font-size", "150%")])]

df_aux.style.set_caption(f'Top {8} Restaurantes').set_table_styles(styles)

<class 'list'>
[33, 31, 38, 16, 20, 67, 74, 25, 78, 4, 249, 273, 218, 177, 191, 230, 317, 271, 197, 166, 321, 337, 607, 619, 1517, 1546, 1515, 632, 366, 580, 1970, 1964, 1991, 768, 760, 769, 790, 782, 793, 779, 690, 694, 649, 674, 1310, 1295, 1316, 1279, 1328, 1485, 2006, 2069, 2064, 2002, 2014, 2067, 2032, 2055, 2020, 2043, 2251, 2258, 2384, 2150, 2207, 2257, 2212, 2216, 2140, 2274, 3533, 3526, 4995, 4067, 2883, 2901, 3192, 3778, 2581, 3077, 5591, 5555, 5581, 5576, 5568, 5580, 5614, 5622, 5604, 5552, 5631, 5691, 5810, 5797, 5715, 5647, 5629, 5644, 5630, 5812, 6123, 6151, 6175, 6114, 6109, 6158, 6142, 6108, 6134, 6173, 6277, 6309, 6286, 6313, 6341, 6342, 6303, 6293, 6312, 6281, 6642, 6393, 6364, 6659, 6651, 6487, 6653, 6534, 6566, 6394, 6709, 6734, 6711, 6733, 6694, 6710, 6708, 6695, 6723, 6732, 6861, 6905, 6908, 6850, 6795, 6794, 6833, 6858, 6894, 6886]
150


,restaurant_id,restaurant_name,country_name,city,cuisines,average_cost_for_two,currency,aggregate_rating,votes
321,16587626,Mandoo Korean Dumplings,Australia,Adelaide,Korean,45,Dollar($),4.900000,730
337,16587684,Andre's Cucina & Polenta Bar,Australia,Adelaide,Italian,100,Dollar($),4.900000,840
607,16592896,Gerard's Bistro,Australia,Brisbane,Middle Eastern,200,Dollar($),4.900000,620
619,16593535,Rogue Bistro,Australia,Brisbane,Modern Australian,140,Dollar($),4.900000,476
1517,16598168,Nobu Perth,Australia,Perth,Japanese,140,Dollar($),4.900000,1822
1546,16598431,Tuck Shop Cafe,Australia,Perth,Modern Australian,55,Dollar($),4.900000,2822
1515,18196379,tbsp.,Australia,Perth,Sandwich,0,Dollar($),4.900000,577
632,18441945,Jacaranda Coffee Lane,Australia,Brisbane,Fusion,50,Dollar($),4.900000,207
366,16588873,Peel St,Australia,Adelaide,Asian,110,Dollar($),4.800000,381
580,16590399,Sono Japanese Restaurant,Australia,Brisbane,Japanese,150,Dollar($),4.800000,1045


## 5.16 - Melhores Restaurantes dos Principais tipos Culinários

In [55]:
culinarias = ['Italian','American','Arabian','Japanese','Brazilian']
colunas = ['restaurant_id', 'restaurant_name','country_name','city','cuisines','average_cost_for_two','currency','aggregate_rating','votes']

inf_culinaria = {}

for culinaria in culinarias:
  #indice_culinaria = dados['cuisines'].isin(culinarias)
  indice_culinaria = dados['cuisines'] == culinaria
  inf_culinaria[culinaria] = dados.loc[indice_culinaria, colunas].sort_values(["aggregate_rating", "restaurant_id"], ascending=[False, True]).iloc[0, :].to_dict()


inf_culinaria


{'Italian': {'restaurant_id': 10021,
  'restaurant_name': 'Darshan',
  'country_name': 'India',
  'city': 'Pune',
  'cuisines': 'Italian',
  'average_cost_for_two': 700,
  'currency': 'Indian Rupees(Rs.)',
  'aggregate_rating': 4.9,
  'votes': 3106},
 'American': {'restaurant_id': 6102616,
  'restaurant_name': 'Burger & Lobster',
  'country_name': 'England',
  'city': 'London',
  'cuisines': 'American',
  'average_cost_for_two': 45,
  'currency': 'Pounds(£)',
  'aggregate_rating': 4.9,
  'votes': 453},
 'Arabian': {'restaurant_id': 18535007,
  'restaurant_name': 'Mandi@36',
  'country_name': 'India',
  'city': 'Hyderabad',
  'cuisines': 'Arabian',
  'average_cost_for_two': 600,
  'currency': 'Indian Rupees(Rs.)',
  'aggregate_rating': 4.7,
  'votes': 1085},
 'Japanese': {'restaurant_id': 6107336,
  'restaurant_name': 'Sushi Samba',
  'country_name': 'England',
  'city': 'London',
  'cuisines': 'Japanese',
  'average_cost_for_two': 110,
  'currency': 'Pounds(£)',
  'aggregate_rating': 4

In [56]:
dados[indice_culinaria]['restaurant_id'].count()

if 'Italian' in inf_culinaria.keys():
  print('esta')
else:
  print('Não está')

inf_culinaria.keys()

lista_culinaria = sorted(list(dados['cuisines'].unique()))

print(lista_culinaria)

esta
['Afghan', 'African', 'American', 'Andhra', 'Arabian', 'Argentine', 'Armenian', 'Asian', 'Asian Fusion', 'Assamese', 'Australian', 'Author', 'Awadhi', 'BBQ', 'Bakery', 'Balti', 'Bar Food', 'Belgian', 'Bengali', 'Beverages', 'Biryani', 'Brazilian', 'Breakfast', 'British', 'Burger', 'Burmese', 'Cafe', 'Cafe Food', 'Cajun', 'California', 'Canadian', 'Cantonese', 'Caribbean', 'Charcoal Chicken', 'Chettinad', 'Chinese', 'Coffee', 'Coffee and Tea', 'Contemporary', 'Continental', 'Creole', 'Crepes', 'Cuban', 'Deli', 'Desserts', 'Dim Sum', 'Dimsum', 'Diner', 'Donuts', 'Drinks Only', 'Durban', 'Döner', 'Eastern European', 'Egyptian', 'European', 'Fast Food', 'Filipino', 'Finger Food', 'Fish and Chips', 'French', 'Fresh Fish', 'Fusion', 'German', 'Giblets', 'Goan', 'Gourmet Fast Food', 'Greek', 'Grill', 'Gujarati', 'Hawaiian', 'Healthy Food', 'Home-made', 'Hyderabadi', 'Ice Cream', 'Indian', 'Indonesian', 'International', 'Iranian', 'Irish', 'Italian', 'Izgara', 'Japanese', 'Juices', 'Kebab